In [1]:
#https://www.kaggle.com/nandhuelan/text-image-metric-learning
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import logging
from datetime import datetime
import math
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, models
from torch import nn
from sentence_transformers.evaluation import BinaryClassificationEvaluator


import cv2
import torch.nn.init as init
import torch
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset
from torchvision import transforms
from torch.optim import Adam, SGD, RMSprop
import time
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
import pickle
import cv2
import torch.nn.functional as F
from torchvision import models as torchmodels
import seaborn as sns
import random
import timm
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

import sys

%matplotlib inline
from pytorch_metric_learning import losses as pml_loss, miners, samplers, trainers, testers
from pytorch_metric_learning.utils import common_functions
import pytorch_metric_learning.utils.logging_presets as logging_presets
import torchvision
from torchvision import datasets, transforms
from PIL import Image
import logging
import matplotlib.pyplot as plt
import umap
from cycler import cycler
import record_keeper
import pytorch_metric_learning
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s"%pytorch_metric_learning.__version__)

INFO:root:VERSION 0.9.97


In [2]:
# What is Metric Learning
"""
The goal of Metric Learning is to learn a representation function that maps objects into an embedded space. The distance in the embedded space should preserve the objects’ similarity — similar objects get close and dissimilar objects get far away. Various loss functions have been developed for Metric Learning.

For example, the contrastive loss guides the objects from the same class to be mapped to the same point and those from different classes to be mapped to different points whose distances are larger than a margin.
Triplet loss is also popular, which requires the distance between the anchor sample and the positive sample to be smaller than the distance between the anchor sample and the negative sample.
"""

'\nThe goal of Metric Learning is to learn a representation function that maps objects into an embedded space. The distance in the embedded space should preserve the objects’ similarity — similar objects get close and dissimilar objects get far away. Various loss functions have been developed for Metric Learning.\n\nFor example, the contrastive loss guides the objects from the same class to be mapped to the same point and those from different classes to be mapped to different points whose distances are larger than a margin.\nTriplet loss is also popular, which requires the distance between the anchor sample and the positive sample to be smaller than the distance between the anchor sample and the negative sample.\n'

In [3]:
"""
train_df=pd.read_csv('../input/shopee-product-matching/train.csv')
test_df=pd.read_csv('../input/shopee-product-matching/test.csv')
siam_data=pd.read_csv('../input/shopee-siamese-training/siamese_data.csv')

train_image_path='../input/shopee-product-matching/train_images/'

siam_data['image_1_path']=siam_data['image_1'].apply(lambda x: train_image_path+x)
siam_data['image_2_path']=siam_data['image_2'].apply(lambda x: train_image_path+x)

train_df['image_path']=train_df['image'].apply(lambda x: train_image_path+x)
labels_list=train_df['label_group'].value_counts().index[:100].tolist() ##filtering only 100 labels

train_df=train_df[train_df['label_group'].isin(labels_list)].reset_index(drop=True)
"""

"\ntrain_df=pd.read_csv('../input/shopee-product-matching/train.csv')\ntest_df=pd.read_csv('../input/shopee-product-matching/test.csv')\nsiam_data=pd.read_csv('../input/shopee-siamese-training/siamese_data.csv')\n\ntrain_image_path='../input/shopee-product-matching/train_images/'\n\nsiam_data['image_1_path']=siam_data['image_1'].apply(lambda x: train_image_path+x)\nsiam_data['image_2_path']=siam_data['image_2'].apply(lambda x: train_image_path+x)\n\ntrain_df['image_path']=train_df['image'].apply(lambda x: train_image_path+x)\nlabels_list=train_df['label_group'].value_counts().index[:100].tolist() ##filtering only 100 labels\n\ntrain_df=train_df[train_df['label_group'].isin(labels_list)].reset_index(drop=True)\n"

In [4]:
# 4.1 Sentence bert finetuning
train_df=pd.read_csv('C:\\Users\\shota\\OneDrive\\ドキュメント\\Data\\Kaggle\\kc_Shopee\\train.csv')
test_df=pd.read_csv('C:\\Users\\shota\\OneDrive\\ドキュメント\\Data\\Kaggle\\kc_Shopee\\test.csv')
siam_data=pd.read_csv('C:\\Users\\shota\OneDrive\\ドキュメント\\Data\Kaggle\\kc_Shopee\\siamese_data.csv')

train_image_path="C:\\Users\\shota\\OneDrive\\ドキュメント\\Data\\Kaggle\\kc_Shopee\\train_images\\"

siam_data['image_1_path'] = siam_data['image_1'].apply(lambda x: train_image_path+x)
siam_data['image_2_path'] = siam_data['image_2'].apply(lambda x: train_image_path+x)
train_df['image_path'] = train_df['image'].apply(lambda x: train_image_path+x)
labels_list=train_df['label_group'].value_counts().index[:100].tolist()

train_df = train_df[train_df['label_group'].isin(labels_list)].reset_index(drop=True)

In [5]:
siam_data.head(2)

,posting_id_1,posting_id_2,title_1,title_2,image_1,image_2,label,image_1_path,image_2_path
0,train_129225211,train_2278313361,Paper Bag Victoria Secret,PAPER BAG VICTORIA SECRET,0000a68812bc7e98c42888dfb1c07da0.jpg,f83b49a86a0ee8592e3bf0204da3fbdf.jpg,1,C:\Users\shota\OneDrive\ドキュメント\Data\Kaggle\kc_...,C:\Users\shota\OneDrive\ドキュメント\Data\Kaggle\kc_...
1,train_129225211,train_866245802,Paper Bag Victoria Secret,BGSR Jaket Cagoule / Jaket Cagoule Premium / J...,0000a68812bc7e98c42888dfb1c07da0.jpg,5b9a1c01bf01447982a8f0a593909d07.jpg,0,C:\Users\shota\OneDrive\ドキュメント\Data\Kaggle\kc_...,C:\Users\shota\OneDrive\ドキュメント\Data\Kaggle\kc_...


In [6]:
siam_data['label'].value_counts()

0    34832
1    20874
Name: label, dtype: int64

In [7]:
"""
#shuffle the data for split
siam_data=siam_data.sample(frac=0.5).reset_index(drop=True) #taking only sample of data
train,val = train_test_split(siam_data, test_size=0.2, random_state=42,
                                                       stratify=siam_data['label'])
                                                       """

"\n#shuffle the data for split\nsiam_data=siam_data.sample(frac=0.5).reset_index(drop=True) #taking only sample of data\ntrain,val = train_test_split(siam_data, test_size=0.2, random_state=42,\n                                                       stratify=siam_data['label'])\n                                                       "

In [8]:
#shuffle the data for split
siam_data=siam_data.sample(frac=0.5).reset_index(drop=True)#taking only sample of data
train, val = train_test_split(siam_data, test_size=0.2, random_state=42, stratify=siam_data['label'])

In [9]:
"""
model_name = 'bert-base-uncased'
train_batch_size = 16
num_epochs = 1
model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
"""

'\nmodel_name = \'bert-base-uncased\'\ntrain_batch_size = 16\nnum_epochs = 1\nmodel_save_path = \'output/training_stsbenchmark_continue_training-\'+model_name+\'-\'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")\n'

In [10]:
model_name = 'bert-base-uncased'
train_batch_size = 16
num_epochs = 1
model_save_path = 'C:\\Users\\shota\\OneDrive\\ドキュメント\\Data\\Kaggle\\output\\training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [11]:
"""
#Get the bert base uncased embedding and pass it into pooling layer and a linear layer for findal representation
word_embedding_model = models.Transformer(model_name, max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=512, activation_function=nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
"""

'\n#Get the bert base uncased embedding and pass it into pooling layer and a linear layer for findal representation\nword_embedding_model = models.Transformer(model_name, max_seq_length=512)\npooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())\ndense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=512, activation_function=nn.Tanh())\n\nmodel = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])\n'

In [12]:
word_embedding_model = models.Transformer(model_name, max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=512, activation_function=nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [13]:
#Prepare dataset
train_samples = []
val_samples = []

for row in train.iterrows():
        inp_example = InputExample(texts=[row[1]['title_1'], row[1]['title_2']], label=row[1]['label'])
        train_samples.append(inp_example)         

for row in val.iterrows():
        inp_example = InputExample(texts=[row[1]['title_1'], row[1]['title_2']], label=row[1]['label'])
        val_samples.append(inp_example)

In [14]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.ContrastiveLoss(model=model)


# Development set: Measure correlation between cosine score and gold labels
logging.info("Read evaluation dev dataset")
evaluator = BinaryClassificationEvaluator.from_input_examples(val_samples, name='contrast-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

INFO:root:Read evaluation dev dataset
INFO:root:Warmup-steps: 140


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1393 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#4.2 Pytorch metric learning(PML) for image embedding